In [1]:
import os

In [2]:
%pwd

'/Users/arash/ML_End_to_End_Pj/end-to-end-solar-dust-detection/research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'/Users/arash/ML_End_to_End_Pj/end-to-end-solar-dust-detection'

## what is Entity?

Its return type of any functions

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzipped_data_dir: Path
    
    


In [6]:
from solar_dust_detection.constants import *
from solar_dust_detection.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath: Path = CONFIG_FILE_PATH,
        params_filepath: Path = PARAMS_FILE_PATH,
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([Path(self.config.artifacts_root)])
        
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        
        data_ingestion_config = DataIngestionConfig(
            root_dir=Path(config.root_dir),
            source_URL=config.source_URL,
            local_data_file=Path(config.local_data_file),
            unzipped_data_dir=Path(config.unzipped_data_dir),
        )
        return data_ingestion_config

In [8]:
import os
import gdown
import zipfile
from solar_dust_detection import logger
from solar_dust_detection.utils.common import get_size

In [15]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_data(self) -> str:
        """Download data from Google Drive."""
        try:
            dataset_url = self.config.source_URL
            zip_download_dir = self.config.local_data_file
            os.makedirs("artifacts/data_ingestion", exist_ok=True)
            logger.info(f"Created directory at: artifacts/data_ingestion")

            file_id = dataset_url.split("/")[-2]
            prefix_url = "https://drive.google.com/uc?export=download&id="
            gdown.download(prefix_url + file_id, str(zip_download_dir))
            logger.info(f"Downloading data from {dataset_url} to {zip_download_dir}")
            logger.info(f"Downloaded file size: {get_size(zip_download_dir)} bytes")

        except Exception as e:
            logger.error(f"Error occurred while downloading data: {e}")
            raise e

    def extract_zip_file(self) -> None:
        """Extract the zip file to the specified directory."""
        unzip_path = self.config.unzipped_data_dir
        os.makedirs(unzip_path, exist_ok=True)

        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
        logger.info(f"Extracted data to {unzip_path}")

## Pipeline

In [16]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_data()
    data_ingestion.extract_zip_file()
except Exception as e:
    logger.error(f"Data ingestion failed: {e}") 
    raise e

[2026-01-23 20:12:59,038: INFO: common]: YAML file: config/config.yaml loaded successfully
[2026-01-23 20:12:59,043: INFO: common]: YAML file: params.yaml loaded successfully
[2026-01-23 20:12:59,044: INFO: common]: Created directory at: artifacts
[2026-01-23 20:12:59,046: INFO: 4137835773]: Created directory at: artifacts/data_ingestion


Downloading...
From (original): https://drive.google.com/uc?export=download&id=1ugZsVw03YafQMvPi7J6E7rD6Tq6jhizz
From (redirected): https://drive.google.com/uc?export=download&id=1ugZsVw03YafQMvPi7J6E7rD6Tq6jhizz&confirm=t&uuid=b9e31675-b864-4de2-b78f-6eb959ba91a4
To: /Users/arash/ML_End_to_End_Pj/end-to-end-solar-dust-detection/artifacts/data_ingestion/solar_dust_detection.zip













































































































































































































































































































































































































































































































































































































































[2026-01-23 20:14:34,297: INFO: 4137835773]: Downloading data from https://drive.google.com/file/d/1ugZsVw03YafQMvPi7J6E7rD6Tq6jhizz/view?usp=sharing to artifacts/data_ingestion/solar_dust_detection.zip
[2026-01-23 20:14:34,301: INFO: common]: File size for artifacts/data_ingestion/solar_dust_detection.zip is 1241599.21 KB
[2026-01-23 20:14:34,301: INFO: 4137835773]: Downloaded file size: 1241599.21 KB bytes


[2026-01-23 20:14:39,934: INFO: 4137835773]: Extracted data to artifacts/data_ingestion


In [ ]:
print(config)